In [7]:
import time
import random
import collections
import numpy as np
from causality_functions import jitter, dependent, simulations_data
from causality_functions import compute_regression, compute_correlation
from causality_functions import add_confidence_stats, confidence_graphs
from causality_functions import stats_graphs

import altair as alt
import pandas as pd
import plotly.express as px
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 40, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 5, 13, 12
Xstd,Ystd = 1, 1

m1, k1 = 1, 0
m2, k2 = 1.5, 0
m3, k3 = 0.15, 0 # this is feedback term
m4, k4 = 0.75, 0.0

dg = 0.1
d1 = 1
d2 = 1
d3 = 1
d4 = 1

errorA = 2
errorB = 4
errorC = 3.5
errorX = 0.75
errorY = 0.6

mrange1 = 0,5 #slope ranges
mrange2 = 0,5
mrange3 = 0,5
mrange4 = 0,5

erangeA = 0,1 #error range
erangeB = 0.2,10 #error range
erangeC = 0.2,10 #error range
erangeX = 0.2,5 #error range
erangeY = 0.2,5 #error range


def linear_positive_C_to_B(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        _, _, C = history[1 - d3]
        _, B, _ = history[-1]
        B_ = jitter(B + A*m1 + C*m3 - B*m2 -dg*B + k1 ,  errorB)
        _, B, _ = history[1 - d2]
        _, _, C = history[-1]
        C_ = jitter(C + B*m2 - dg*C + k2, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def linear_positive_A_to_C(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        B_ = dependent(A, m1, k1 ,  errorB)
        _, B, _ = history[1 - d2]
        A, _, _ = history[1 - d1]
        _, _, C = history[-1]
        C_ = jitter(A*m3+ B*m2+ -dg*C + k2, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])

def update_slopes_errors():
    def select(lower, upper):
        return lower + random.random()*(upper-lower)
    
    global m1, m2, m3, m4
    global errorA, errorB, errorC, errorX, errorY

    m1 = select(*mrange1)
    m2 = select(*mrange2)
    m3 = select(*mrange3)
    m4 = select(*mrange4)
    
    errorA = select(*erangeA)
    errorB = select(*erangeB)
    errorC = select(*erangeC)
    errorX = select(*erangeX)
    errorY = select(*erangeY)
    
def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        update_slopes_errors()
        ABC = simulations_data(linear_positive_A_to_C, samplesize) # you can change pathway here
        r = compute_regression(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [2]:
add_confidence_stats(d, ABC_all)
confidence_graphs(d)

alt.VConcatChart(...)

In [3]:
stats_graphs(d)

alt.VConcatChart(...)

In [4]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")

alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [5]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,...,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence,confidence_residual_corr,confidence_corrected_bc_corr,confidence_slope_AC
0,4.417231e+37,0.000000e+00,2.045933e+37,9.403468e+35,0.463171,4.355414e+35,0.470574,1.0,0.470574,-0.044527,...,0.685984,1.0,0.685984,0.000000e+00,-0.999854,-1.475740e+20,within,within,,within
1,4.113147e+66,0.000000e+00,-3.714642e+66,8.023623e+64,-0.903114,-7.246249e+64,0.160221,1.0,0.160221,-0.570660,...,0.400276,-1.0,-0.400276,-1.110223e-16,-0.994379,3.507604e+49,within,more,less,within
2,1.980542e+27,3.665772e+15,3.168924e+27,3.156494e+25,1.600028,5.050480e+25,0.358974,1.0,0.358974,-0.016980,...,0.599144,1.0,0.599144,-3.796963e-13,-0.998986,-2.667175e+13,within,within,,within
3,1.154736e+52,-5.316912e+36,-9.986305e+51,1.237801e+50,-0.864813,-1.070466e+50,0.005689,1.0,0.005689,-0.719608,...,0.075422,-1.0,-0.075422,1.734723e-18,-0.993216,2.076919e+34,within,more,less,within
4,2.652495e+04,-1.192360e+01,7.510259e+04,2.532825e+03,2.830477,7.168201e+03,0.629416,1.0,0.629258,0.273487,...,0.793358,1.0,0.793258,1.583206e-04,-0.999994,9.016987e-01,within,less,less,within
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9.809192e+24,-2.867788e+15,1.873965e+25,1.400572e+23,1.910417,2.675676e+23,0.334850,1.0,0.334850,-0.022091,...,0.578662,1.0,0.578662,-2.202158e-10,-0.999975,-8.798371e+13,within,within,,within
96,-7.244437e+69,0.000000e+00,3.900497e+69,-2.376160e+68,-0.538413,1.279355e+68,0.515645,1.0,0.515645,0.023417,...,-0.718085,-1.0,0.718085,5.551115e-16,-0.999917,2.394524e+52,within,within,,within
97,-1.542342e+68,9.578097e+52,1.174036e+68,-7.978047e+66,-0.761203,6.072913e+66,0.648847,1.0,0.648847,0.026379,...,-0.805511,-1.0,0.805511,7.771561e-16,-0.992737,3.741444e+51,within,within,,within
98,1.454957e+31,-6.755399e+15,9.121348e+30,1.786138e+29,0.626915,1.119757e+29,0.308605,1.0,0.308605,-0.113600,...,0.555522,1.0,0.555522,-3.330669e-16,-0.997605,3.518437e+13,within,more,,within
